In [0]:
%run /Workspace/Users/njain210899@gmail.com/HealthcareRepo/landing-layer/01-config/

In [0]:
config  = config()

In [0]:
from pyspark.sql.functions import lit
dept_hosa_df = spark.read.parquet(config.get_bronze_volume_dir()+'/departments/hos-a')
dept_hosa_df = dept_hosa_df.withColumn('datasource',lit('hos-a'))
dept_hosb_df = spark.read.parquet(config.get_bronze_volume_dir()+'/departments/hos-b')
dept_hosb_df = dept_hosb_df.withColumn('datasource',lit('hos-b'))


In [0]:
df_merged = dept_hosa_df.unionByName(dept_hosb_df)

In [0]:
from pyspark.sql.functions import col, concat
df_merged = df_merged.withColumn('SRC_Dept_Id',col('DeptId'))\
    .withColumn('DeptID',concat(col('DeptId'),lit('-'),col('datasource')))

In [0]:
df_merged.columns

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.departments
(
  SRC_DEPT_ID STRING,
  DEPT_ID STRING,
  DEPT_NAME STRING,
  DATASOURCE STRING,
  is_quarantined BOOLEAN
)
USING DELTA

In [0]:
%sql
truncate table silver.departments

In [0]:
df_merged.createOrReplaceTempView('vw_Dept')

In [0]:
%sql
insert into silver.departments
select SRC_Dept_Id,DeptID, Name, datasource,
case when src_dept_id is null or name is null then True else False end 
from vw_Dept

In [0]:
%sql
select * from silver.departments